In [0]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import tensorflow as tf
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

tf.test.gpu_device_name()


In [0]:
#!cp -r /content/drive/My\ Drive/ComputerVisionHackathon/data /content/sample_data/
!rm -rf /content/sample_data/data/train/.ipynb_checkpoints

In [0]:
# I want to check if all the images are of same sizes.

import os
from PIL import Image 

folder_images = "/content/sample_data/data/train/emer/_TO_STOP_THIS_CELL"
size_images = dict()

for dirpath, _, filenames in os.walk(folder_images):
    print("How many execute")
    for path_image in filenames:
        image = os.path.abspath(os.path.join(dirpath, path_image))
        with Image.open(image) as img:
            width, heigth = img.size
            size_images[path_image] = {'width': width, 'heigth': heigth}
            if width != 224 or heigth != 224:
              print(width,heigth,path_image)
            else:
              sys.stdout.write('-')

#print(size_images)

In [0]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.25))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.3))
	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.22))
	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.25))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	#opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
	return model



In [0]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.show()


In [0]:
%%timeit
# run the test harness for evaluating a model
def run_test_harness():
  # define model
  model = define_model()
  # create data generators
  #datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
  #test_datagen = ImageDataGenerator(rescale=1.0/255.0)
  # prepare iterators
  #train_it = train_datagen.flow_from_directory('/content/sample_data/data/train', 
  #                                             class_mode='binary', batch_size=64, target_size=(200, 200))
  #test_it = test_datagen.flow_from_directory('/content/sample_data/data/test_root',
  #                                           class_mode=None, batch_size=64, target_size=(200, 200))
  
  TRAIN_DIR = '/content/sample_data/data/train/'
  
  #datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
  datagen = ImageDataGenerator(validation_split=0.2,
                               rescale=1.0/255.0, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1,
                               horizontal_flip=True
                               )
  
  train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    subset='training',
                              color_mode="rgb", 
                              class_mode='binary',
                              shuffle=True,
                              batch_size=32,
                              target_size=(224, 224),
                              seed=123
    #subset='training',shuffle=True,
    #class_mode='binary', batch_size=64, target_size=(224, 224)
    )
  labels = (train_generator.class_indices)
  
  print(labels)

  val_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
                              color_mode="rgb", 
                              class_mode='binary',
                              shuffle=True,
                              batch_size=32,
                              target_size=(224, 224),
                              seed=123
    #shuffle=True,
    #class_mode='binary', batch_size=64, target_size=(224, 224)
    )

  # fit model
  history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator),
                                validation_data=val_generator, validation_steps=len(val_generator), epochs=500,verbose=2)
  model.save('final_model.h6')
  #_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=2)
  
  #print('> %.3f' % (acc * 100.0))
  # learning curves
  summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

In [0]:
#import cv2
#import numpy as np
#from matplotlib import pyplot as plt

#img = cv2.imread("/content/ipykernel_launcher.py_plot.png")
#img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#plt.imshow(img_cvt)
#plt.show()

In [0]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

model = load_model('final_model.h6')

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	#img = img - [123.68, 116.779, 103.939]
	return img

# load an image and predict the class
def run_example(pic):
	# load the image
	img = load_image(pic)
	print (img.shape)
 
	# load model
	#model = load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	return int(result[0])



In [0]:
x = run_example("/content/sample_data/data/test_root/test/100.jpg")
print (x)

In [0]:
def show_test_image(pic):
  img = cv2.imread(pic)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img_cvt)
  ttl = run_example(pic)
  plt.title(ttl)
  plt.show()

In [0]:
# entry point, run the example
#run_example()

import pandas as pd
test_path ="/content/sample_data/data/test_root/test/"

testdf = pd.read_csv("/content/drive/My Drive/ComputerVisionHackathon/test.csv")
#testdf = testdf[357:]
testdf


In [0]:
df = pd.DataFrame(columns=['image_names','emergency_or_not'])
test_img_name = testdf['image_names'].tolist()
for i,photo in enumerate(test_img_name):
  pred = run_example(test_path+photo)
  print(i,photo,pred)
  df.loc[i] = [photo,pred]
df

In [0]:
df

In [0]:
df.to_csv("Subimission_10.csv", index=False)
#df.to_csv('Subimission_six.csv', mode='a', header=False)